In [1]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('tmdb_5000_movies.csv', delimiter=',')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4798 entries, 0 to 4797
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4798 non-null   int64  
 1   genres                4798 non-null   object 
 2   homepage              1711 non-null   object 
 3   id                    4798 non-null   int64  
 4   keywords              4798 non-null   object 
 5   original_language     4798 non-null   object 
 6   original_title        4798 non-null   object 
 7   overview              4798 non-null   object 
 8   popularity            4798 non-null   float64
 9   production_companies  4798 non-null   object 
 10  production_countries  4798 non-null   object 
 11  release_date          4797 non-null   object 
 12  revenue               4798 non-null   int64  
 13  runtime               4798 non-null   int64  
 14  spoken_languages      4798 non-null   object 
 15  status               

In [3]:
df.shape

(4798, 22)

In [4]:
from ast import literal_eval

# Specifying the desired features list and applying literal_eval function to it

features = ['cast', 'crew', 'keywords', 'genres', 'production_companies']

for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [5]:
df[feature]

0       [{'name': 'Ingenious Film Partners', 'id': 289...
1       [{'name': 'Walt Disney Pictures', 'id': 2}, {'...
2       [{'name': 'Columbia Pictures', 'id': 5}, {'nam...
3       [{'name': 'Legendary Pictures', 'id': 923}, {'...
4             [{'name': 'Walt Disney Pictures', 'id': 2}]
                              ...                        
4793             [{'name': 'Columbia Pictures', 'id': 5}]
4794                                                   []
4795    [{'name': 'Front Street Pictures', 'id': 3958}...
4796                                                   []
4797    [{'name': 'rusty bear entertainment', 'id': 87...
Name: production_companies, Length: 4798, dtype: object

In [6]:
def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
    return np.nan

# 2. Getting list top 3 elements or, entire list if elements < 3 (whichever is more)
# Specifying the desired features list and applying literal_eval function to it
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
#        if len(names) > 3: # checking whether more than 3 elements exist 
#           names = names[:3] # & if yes, then returning only the 1st 3 ekements. else, returning the entire list
        return names
    return [] # returning empty list incase of missing or malformed data

In [7]:
df['director'] = df['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres', 'production_companies']

for feature in features:
    df[feature] = df[feature].apply(get_list)

In [8]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
    # checking whether director exists. If not, returning empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ",""))
        else:
            return ''

In [9]:
features = ['cast', 'keywords', 'director', 'genres', 'production_companies']

for feature in features:
    df[feature] = df[feature].apply(clean_data)

In [10]:
df[['title', 'cast', 'director', 'keywords', 'genres', 'production_companies']].head(10)

,title,cast,director,keywords,genres,production_companies
0,Avatar,"[samworthington, zoesaldana, sigourneyweaver, ...",jamescameron,"[cultureclash, future, spacewar, spacecolony, ...","[action, adventure, fantasy, sciencefiction]","[ingeniousfilmpartners, twentiethcenturyfoxfil..."
1,Pirates of the Caribbean: At World's End,"[johnnydepp, orlandobloom, keiraknightley, ste...",goreverbinski,"[ocean, drugabuse, exoticisland, eastindiatrad...","[adventure, fantasy, action]","[waltdisneypictures, jerrybruckheimerfilms, se..."
2,Spectre,"[danielcraig, christophwaltz, léaseydoux, ralp...",sammendes,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[action, adventure, crime]","[columbiapictures, danjaq, b24]"
3,The Dark Knight Rises,"[christianbale, michaelcaine, garyoldman, anne...",christophernolan,"[dccomics, crimefighter, terrorist, secretiden...","[action, crime, drama, thriller]","[legendarypictures, warnerbros., dcentertainme..."
4,John Carter,"[taylorkitsch, lynncollins, samanthamorton, wi...",andrewstanton,"[basedonnovel, mars, medallion, spacetravel, p...","[action, adventure, sciencefiction]",[waltdisneypictures]
5,Spider-Man 3,"[tobeymaguire, kirstendunst, jamesfranco, thom...",samraimi,"[dualidentity, amnesia, sandstorm, loveofone's...","[fantasy, action, adventure]","[columbiapictures, lauraziskinproductions, mar..."
6,Tangled,"[zacharylevi, mandymoore, donnamurphy, ronperl...",byronhoward,"[hostage, magic, horse, fairytale, musical, pr...","[animation, family]","[waltdisneypictures, waltdisneyanimationstudios]"
7,Avengers: Age of Ultron,"[robertdowneyjr., chrishemsworth, markruffalo,...",josswhedon,"[marvelcomic, sequel, superhero, basedoncomicb...","[action, adventure, sciencefiction]","[marvelstudios, primefocus, revolutionsunstudios]"
8,Harry Potter and the Half-Blood Prince,"[danielradcliffe, rupertgrint, emmawatson, tom...",davidyates,"[witch, magic, broom, schoolofwitchcraft, wiza...","[adventure, fantasy, family]","[warnerbros., heydayfilms]"
9,Batman v Superman: Dawn of Justice,"[benaffleck, henrycavill, galgadot, amyadams, ...",zacksnyder,"[dccomics, vigilante, superhero, basedoncomicb...","[action, adventure, fantasy]","[dccomics, atlasentertainment, warnerbros., dc..."


In [11]:
Overview_Keywords = []

for index, row in df.iterrows():
    overview = row['overview']
    
    # instantiating Rake, by default uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(overview)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    Overview_Keywords.append(list(key_words_dict_scores.keys()))
    
df['Overview_Keywords'] = Overview_Keywords

# dropping the overview column
df.drop(columns = ['overview'], inplace = True)

In [12]:
df.set_index('title', inplace = True)
df[['cast', 'director', 'keywords', 'genres', 'production_companies', 'Overview_Keywords']].head()

,cast,director,keywords,genres,production_companies,Overview_Keywords
title,,,,,,
Avatar,"[samworthington, zoesaldana, sigourneyweaver, ...",jamescameron,"[cultureclash, future, spacewar, spacecolony, ...","[action, adventure, fantasy, sciencefiction]","[ingeniousfilmpartners, twentiethcenturyfoxfil...","[dispatched, protecting, 22nd, century, follow..."
Pirates of the Caribbean: At World's End,"[johnnydepp, orlandobloom, keiraknightley, ste...",goreverbinski,"[ocean, drugabuse, exoticisland, eastindiatrad...","[adventure, fantasy, action]","[waltdisneypictures, jerrybruckheimerfilms, se...","[earth, captain, barbossa, quite, life, long, ..."
Spectre,"[danielcraig, christophwaltz, léaseydoux, ralp...",sammendes,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[action, adventure, crime]","[columbiapictures, danjaq, b24]","[sinister, organization, past, sends, secret, ..."
The Dark Knight Rises,"[christianbale, michaelcaine, garyoldman, anne...",christophernolan,"[dccomics, crimefighter, terrorist, secretiden...","[action, crime, drama, thriller]","[legendarypictures, warnerbros., dcentertainme...","[reputation, overwhelms, gotham, death, protec..."
John Carter,"[taylorkitsch, lynncollins, samanthamorton, wi...",andrewstanton,"[basedonnovel, mars, medallion, spacetravel, p...","[action, adventure, sciencefiction]",[waltdisneypictures],"[reluctantly, becomes, embroiled, people, rest..."


In [13]:
def create_soup(x): # minimalistic corpus of text documents
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) + ' ' + ' '.join(x['production_companies']) + ' ' + ' '.join(x['Overview_Keywords'])
df['soup'] = df.apply(create_soup, axis=1)

In [14]:
df[['soup']]

,soup
title,
Avatar,cultureclash future spacewar spacecolony socie...
Pirates of the Caribbean: At World's End,ocean drugabuse exoticisland eastindiatradingc...
Spectre,spy basedonnovel secretagent sequel mi6 britis...
The Dark Knight Rises,dccomics crimefighter terrorist secretidentity...
John Carter,basedonnovel mars medallion spacetravel prince...
...,...
El Mariachi,unitedstates–mexicobarrier legs arms paperknif...
Newlyweds,edwardburns kerrybishé marshadietlein caitlin...
"Signed, Sealed, Delivered",date loveatfirstsight narration investigation ...


In [16]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['soup'])

In [17]:
# Cosine similarity is a method to measure the difference between two non-zero vectors of an inner product space

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [18]:
# list used later to match the indexes
indices = pd.Series(df.index)
indices[:10]

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
5                                Spider-Man 3
6                                     Tangled
7                     Avengers: Age of Ultron
8      Harry Potter and the Half-Blood Prince
9          Batman v Superman: Dawn of Justice
Name: title, dtype: object

In [19]:
# function that takes in movie title as input and returns the top 10 recommended movies
def get_recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [23]:
get_recommendations('The Fast and the Furious')

['Fast Five',
 '2 Fast 2 Furious',
 'Stone Cold',
 'Point Break',
 'S.W.A.T.',
 'Furious 7',
 'The Glimmer Man',
 'The Fast and the Furious: Tokyo Drift',
 'Half Past Dead',
 'Alex Cross']